## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.319,73.40,88,75,4.61,broken clouds
1,1,Saint Anthony,US,45.0205,-93.218,39.99,41,90,10.36,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.300,48.20,61,20,9.22,few clouds
3,3,Santa Marta,CO,11.2408,-74.199,98.01,52,66,1.01,broken clouds
4,4,Vaini,TO,-21.2000,-175.200,73.40,94,20,3.44,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Ugoofaaru,MV,5.6667,73.0000,83.53,73,84,9.98,broken clouds
11,11,Xilitla,MX,21.3333,-98.9667,75.79,58,92,6.53,overcast clouds
16,16,Pisco,PE,-13.7000,-76.2167,75.20,64,0,18.41,clear sky
19,19,Saint-Philippe,RE,-21.3585,55.7679,77.00,88,75,6.91,light rain
20,20,Avarua,CK,-21.2078,-159.7750,77.00,73,20,4.61,few clouds
22,22,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,40,16.11,scattered clouds
24,24,Saint-Paul,RE,-21.0096,55.2707,77.00,83,75,6.91,broken clouds
29,29,Sao Miguel,BR,-5.4667,-35.3667,87.80,66,40,16.11,scattered clouds
40,40,Victoria,HK,22.2855,114.1577,75.00,82,31,14.76,scattered clouds
42,42,Mahebourg,MU,-20.4081,57.7000,78.80,89,75,6.53,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                224
City                   224
Country                219
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng",]].copy() 
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Ugoofaaru,MV,83.53,broken clouds,5.6667,73.0000,
11,Xilitla,MX,75.79,overcast clouds,21.3333,-98.9667,
16,Pisco,PE,75.20,clear sky,-13.7000,-76.2167,
19,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,
20,Avarua,CK,77.00,few clouds,-21.2078,-159.7750,
22,Puerto Ayora,EC,82.40,scattered clouds,-0.7393,-90.3518,
24,Saint-Paul,RE,77.00,broken clouds,-21.0096,55.2707,
29,Sao Miguel,BR,87.80,scattered clouds,-5.4667,-35.3667,
40,Victoria,HK,75.00,scattered clouds,22.2855,114.1577,
42,Mahebourg,MU,78.80,broken clouds,-20.4081,57.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Ugoofaaru,MV,83.53,broken clouds,5.6667,73.0000,Ungoofaaru Inn
11,Xilitla,MX,75.79,overcast clouds,21.3333,-98.9667,Rio Tancuilin
16,Pisco,PE,75.20,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
19,Saint-Philippe,RE,77.00,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
20,Avarua,CK,77.00,few clouds,-21.2078,-159.7750,Paradise Inn


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   219
Country                219
Max Temp               219
Current Description    219
Lat                    219
Lng                    219
Hotel Name             219
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))